In [1]:
import sys
sys.path.append('/Users/nissani/Desktop/Hateful_Memes_Project/LoadingData')
sys.path.append('/Users/nissani/Desktop/Hateful_Memes_Project/FeatureGeneration')
import os
import LoadingData
from FER_featurizer import FER_Wrapper
from hate_speech import HateWrapper
from sentence_encoder import SentenceTransformer
from flair.models import TextClassifier
from flair.data import Sentence
import torch
import torchvision

Instructions for updating:
non-resource variables are not supported in the long term


25-08-2020:12:23:01,736 INFO     [file_utils.py:39] PyTorch version 1.4.0 available.
25-08-2020:12:23:01,737 INFO     [file_utils.py:55] TensorFlow version 2.2.0 available.


In [2]:
train_data = LoadingData.LoadingData('/Users/nissani/Desktop/Hateful_Memes_Project/data/train.jsonl')
dev_data = LoadingData.LoadingData('/Users/nissani/Desktop/Hateful_Memes_Project/data/dev.jsonl')
image_features = LoadingData.LoadingData('/Users/nissani/Desktop/Hateful_Memes_Project/data/cleaned_getty_data.csv')

In [3]:
train_data = train_data.load_data()

In [4]:
dev_data = dev_data.load_data()

In [5]:
image_features = image_features.load_data()
image_features = image_features.drop('Unnamed: 0', axis = 1)

In [12]:
def create_features(data):
    FER = FER_Wrapper()
    hate = HateWrapper()
    sentiment_classifier = TextClassifier.load('en-sentiment')
    
    aggregate_text = []
    for el in data:
        aggregate_text.append(el['text'])
    
    for el in data:
        path = '/Users/nissani/Desktop/Hateful_Memes_Project/data/' + el['img']
        emotion_feature = FER.run_FER(path)
        img_id = el['img'].split('/')[1]
        el['emotion_feature'] = emotion_feature[img_id]

    hate_scores = hate.predict(aggregate_text)
    for el in data:
        el['hate_speech'] = hate_scores[hate_scores.text == el['text']].hate_speech
        el['offensive_language'] = hate_scores[hate_scores.text == el['text']].offensive_language
        el['neither'] = hate_scores[hate_scores.text == el['text']].neither

    for el in data:
        sentence = Sentence(el['text'])
        sentiment_classifier.predict(sentence)
        label = str(sentence.labels[0]).split(' ')[0]
        proba = float(str(sentence.labels[0]).split(' ')[1].replace('(', '').replace(')', ''))
        el['sentiment'] = [label, proba]
        
    return data

In [13]:
featurized_train_data = create_features(train_data)

/usr/local/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


2020-08-25 15:44:31,714 loading file /Users/nissani/.flair/models/sentiment-en-mix-distillbert.pt


/usr/local/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
25-08-2020:15:44:31,987 INFO     [configuration_utils.py:265] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at /Users/nissani/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
25-08-2020:15:44:31,988 INFO     [configuration_utils.py:301] Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  